In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Scrape the Wikipedia Page
website_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

content = requests.get(website_url).text
soup = BeautifulSoup(content, 'lxml')
codes_html = soup.find('table', {'class':'wikitable sortable'})

table_rows = codes_html.find_all('tr')

df_rows = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text.strip() for i in td]
    if row != []:
        df_rows += [row]

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(df_rows, columns=['Postcode', 'Borough', 'Neighborhood'])

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.    
df = df[df['Borough'] != 'Not assigned']

#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df = df.groupby(by=['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df['Neighborhood'] = df.apply(lambda x: x['Borough'] if x['Neighborhood']=='Not assigned' else x['Neighborhood'], axis=1)

# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe
df.shape

(103, 3)

In [43]:
# Use the Geocoder package or the csv file to create the following dataframe:
df2 = pd.read_csv('Geospatial_Coordinates.csv')
df3 = pd.merge(df, df2, on='Postcode')
df3

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [51]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [53]:
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto